In [ ]:
import gmsh
import sys

# Adaptive Mesh Generation 

In [ ]:


gmsh.initialize()
gmsh.model.add("Fiber_Composite")

#2R<spacing to avoid overlap of fibers

L = 1.0     # Domain size
R = 0.05    # Fiber radius  
lc = 0.03   # Mesh resolution (Balance between accuracy and condition number)
occ = gmsh.model.occ
  

# Calculate max allowable radius to prevent touching each other or the wall
# R must be less than spacing/2
R = 0.035
# Create Geometry
square = occ.addRectangle(0, 0, 0, L, L) #Rectangle(x, y, z, width, height)
# Create multiple fibers in a grid pattern
fiber_list = []
n_fibers_x = 10
n_fibers_y = 10
spacing = L / n_fibers_x

for i in range(n_fibers_x): # Loop over number of fibers in x direction 
    for j in range(n_fibers_y): #for each x fiber, loop over number of fibers in y direction
                                    x_c = (i+0.5) * spacing #center of fiber in x direction. Note we add 0.5 to center to make sure its in middle of spacing. we don't want circle at edge of spacing
                                    y_c = (j+0.5) * spacing
                                    # Create disk and Both circle and square start from same plane z=0. it uses the center of the circle as the anchor point, but that center is calculated relative to the origin (0,0) of the square (Bottom-Left Corner) .
                                    c = occ.addDisk(x_c, y_c, 0, R, R) #Disk(x_center, y_center, z_center, radius_x, radius_y)  
                                    fiber_list.append((2, c))




# Fragment and Synchronize
# ov contains all resulting surfaces after the boolean operation
ov, ovv = occ.fragment([(2, square)], fiber_list) #Fragment the square with the circle {dim, tag}. Note ovv are separate entities
occ.synchronize()

#  Dynamically Identify Surfaces
#The problem is that after the fragmenting.Gmsh doesn't inherently know which ID belongs to the "Matrix" and which belongs to the "Fiber." It just sees two surfaces.
# We look for the surface whose center of mass is the center of the square. # 

fiber_surface = []
matrix_surface = []

for surface in gmsh.model.getEntities(2):
# Get mass properties (this returns [mass/area, x, y, z])
# For a 2D surface, 'mass' is actually the area.
    mass_center = occ.getCenterOfMass(surface[0], surface[1])

    # We use a tool to get the actual area
    # (Dimension 2, ID surface[1])
    mass_props = occ.getMass(surface[0], surface[1])
    area = mass_props 

    # Logic: If it's small, it's the fiber. If it's large, it's the matrix.
    expected_fiber_area = 3.14159 * (R**2)

    if abs(area - expected_fiber_area) < 1e-4:
        fiber_surface.append(surface[1])
        print(f"Surface {surface[1]} identified as FIBER (Area: {area:.4f})")
    else:
        matrix_surface.append(surface[1])
        print(f"Surface {surface[1]} identified as MATRIX (Area: {area:.4f})")

# Now we have the IDs for both surfaces.
#Assign Physical Groups for Material Domains to help solver recognize them
gmsh.model.addPhysicalGroup(2, fiber_surface, tag=1, name="Fiber_Domain")
gmsh.model.addPhysicalGroup(2, matrix_surface, tag=2, name="Matrix_Domain")

 #Identify and Tag Boundary Edges ---
left_edges = []
right_edges = []
top_edges = []
bottom_edges = []
fiber_interface = []
# Get all 1D entities (curves/lines)
for edge in gmsh.model.getEntities(1):
# Get the bounding box of the line [xmin, ymin, zmin, xmax, ymax, zmax]
    bounds = gmsh.model.getBoundingBox(edge[0], edge[1])
# Unpack bounds  
    xmin, ymin, xmax, ymax = bounds[0], bounds[1], bounds[3], bounds[4] #[xmin, ymin, zmin, xmax, ymax, zmax]

# Check position with a small tolerance (1e-6)
if abs(xmin) < 1e-6 and abs(xmax) < 1e-6:
    left_edges.append(edge[1])                       #append edge ID to left_edges
elif abs(xmin - L) < 1e-6 and abs(xmax - L) < 1e-6:
    right_edges.append(edge[1])                      #append edge ID to right_edges
elif abs(ymin) < 1e-6 and abs(ymax) < 1e-6:
    bottom_edges.append(edge[1])                     #append edge ID to bottom_edges
elif abs(ymin - L) < 1e-6 and abs(ymax - L) < 1e-6:
    top_edges.append(edge[1])                        #append edge ID to top_edges
else:
# If it's not an outer wall, it's the interface between fiber and matrix!
    fiber_interface.append(edge[1])

# Create Physical Group for the interface (Tag 20)
gmsh.model.addPhysicalGroup(1, fiber_interface, tag=20, name="Fiber_Interface")
# Create Physical Groups for the boundaries
gmsh.model.addPhysicalGroup(1, left_edges, tag=10, name="Left_Side") ## (dimension, list_of_ids, tag, name)
gmsh.model.addPhysicalGroup(1, right_edges, tag=11, name="Right_Side")
gmsh.model.addPhysicalGroup(1, top_edges, tag=12, name="Top_Side")
gmsh.model.addPhysicalGroup(1, bottom_edges, tag=13, name="Bottom_Side")

# gmsh.model.mesh.setSize(gmsh.model.getEntities(0), lc) #Set mesh size at all vertices (0D entities) smaller lc = finer mesh(so its saying keep the distance between any two triangles at corner points of geometry  smaller in other words make the mesh smaller)
# gmsh.model.mesh.generate(2) #Generate 2D mesh 2 is dimension


# Local Mesh Refinement (Fields)  

# FIELD 1: Distance - Calculates distance from every point to the fiber edges
f1 = gmsh.model.mesh.field.add("Distance")
gmsh.model.mesh.field.setNumbers(f1, "CurvesList", fiber_interface)

# FIELD 2: Threshold - Takes the distance and maps it to a mesh size
f2 = gmsh.model.mesh.field.add("Threshold")
gmsh.model.mesh.field.setNumber(f2, "InField", f1)      # Use the distance field as input
gmsh.model.mesh.field.setNumber(f2, "SizeMin", lc / 6)  # Mesh size AT the fiber edge (tiny;closer to fiber)
gmsh.model.mesh.field.setNumber(f2, "SizeMax", lc)      # Mesh size in the matrix (normal)
gmsh.model.mesh.field.setNumber(f2, "DistMin", 0.01)    # Range for SizeMin (all points closer than this distance get SizeMin)
gmsh.model.mesh.field.setNumber(f2, "DistMax", 0.15)    # Distance where it reaches SizeMax (all points farther than this distance get SizeMax)

# FIELD 3: Constant - This ensures the mesh stays small INSIDE the fiber surfaces
f3 = gmsh.model.mesh.field.add("Constant") 
gmsh.model.mesh.field.setNumbers(f3, "SurfacesList", fiber_surface) #F3 returns default mesh value if point is outside fibers 
gmsh.model.mesh.field.setNumber(f3, "VIn", lc / 6)      # Force this size inside the fibers

# FIELD 4: Min - Combines f2 (matrix transition) and f3 (fiber interior)
# It tells Gmsh: "At any point, use the SMALLEST size from either Field 2 or Field 3"
f_min = gmsh.model.mesh.field.add("Min")
gmsh.model.mesh.field.setNumbers(f_min, "FieldsList", [f2, f3])

# Activate the refinement using the combined logic
gmsh.model.mesh.field.setAsBackgroundMesh(f_min) #this combines both fields so that we have fine mesh near fiber edges and also fine mesh inside fibers. Check f2 and f3 for details. 

# Disable default settings that might override our fields
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)

#  Generate
gmsh.model.mesh.generate(2)
gmsh.write("refined_composite_with_interior.msh")

if '-nopopup' not in sys.argv:
 gmsh.fltk.run()

gmsh.finalize()

Surface 2 identified as FIBER (Area: 0.0038)                                                                                                  
Surface 3 identified as FIBER (Area: 0.0038)
Surface 4 identified as FIBER (Area: 0.0038)
Surface 5 identified as FIBER (Area: 0.0038)
Surface 6 identified as FIBER (Area: 0.0038)
Surface 7 identified as FIBER (Area: 0.0038)
Surface 8 identified as FIBER (Area: 0.0038)
Surface 9 identified as FIBER (Area: 0.0038)
Surface 10 identified as FIBER (Area: 0.0038)
Surface 11 identified as FIBER (Area: 0.0038)
Surface 12 identified as FIBER (Area: 0.0038)
Surface 13 identified as FIBER (Area: 0.0038)
Surface 14 identified as FIBER (Area: 0.0038)
Surface 15 identified as FIBER (Area: 0.0038)
Surface 16 identified as FIBER (Area: 0.0038)
Surface 17 identified as FIBER (Area: 0.0038)
Surface 18 identified as FIBER (Area: 0.0038)
Surface 19 identified as FIBER (Area: 0.0038)
Surface 20 identified as FIBER (Area: 0.0038)
Surface 21 identified as FIBER (Area